In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
import sys
import re
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gc
from sklearn.model_selection import train_test_split


In [27]:
df = pd.read_csv("./data/Latest_Data_Science_Salaries.csv",sep=";")


In [28]:
from sklearn.model_selection import train_test_split

# splitting data
# X = df.drop(["Salary in USD","Year","Salary"], 1)
# y = df[["Salary in USD"]]

train, test = train_test_split(df, test_size=0.20,shuffle=False)

In [29]:
train.head()

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023


In [30]:
test.head()

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
2640,Data Engineer,Full-Time,Executive,Director,260000,United States Dollar,United States,260000,United States,Medium,2022
2641,Machine Learning Developer,Full-Time,Entry,Junior,15000,United States Dollar,Thailand,15000,Thailand,Large,2021
2642,Data Engineer,Full-Time,Senior,Expert,193000,United States Dollar,United States,193000,Spain,Medium,2022
2643,Data Scientist,Full-Time,Entry,Junior,93000,United States Dollar,United States,93000,United States,Medium,2022
2644,Data Scientist,Full-Time,Entry,Junior,73000,United States Dollar,United States,73000,United States,Medium,2022


In [31]:
train_test = pd.concat([train, test], axis=0)
train_test.head()

,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023


In [32]:
sys.setrecursionlimit(100000)
begin = time.time()
full_texts = []
for i in range(len(train_test)):
    item = train_test.iloc[i]
    full_text = "Job Title %s Employment Type %s Experience Level %s Expertise Level %s"%(str(item["Job Title"]), str(item["Employment Type"]), str(item["Experience Level"]), str(item["Expertise Level"]))
    full_texts.append(full_text)
print("Elapsed time: %.2fs"%(time.time() - begin))

Elapsed time: 0.47s


In [33]:
train_test["full_text"] = full_texts

In [34]:
del full_texts
gc.collect()

6449

In [35]:
def preprocess(df):
    df['full_text'] = df['full_text'].str.replace(r'[^\w\s]+', '')
    df['full_text'] = df['full_text'].str.lower()
    df["full_text_tokens"] = df["full_text"].apply(lambda item: item.split(" "))
    df["full_text_sequence_length"] = df["full_text_tokens"].apply(lambda item: len(item))
    return df

In [36]:
%%time
train_test = preprocess(train_test)
train_test.head()

CPU times: total: 15.6 ms
Wall time: 32.8 ms


C:\Users\oueo5\AppData\Local\Temp\ipykernel_9844\3661290852.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['full_text'] = df['full_text'].str.replace(r'[^\w\s]+', '')


,Job Title,Employment Type,Experience Level,Expertise Level,Salary,Salary Currency,Company Location,Salary in USD,Employee Residence,Company Size,Year,full_text,full_text_tokens,full_text_sequence_length
0,Data Engineer,Full-Time,Senior,Expert,210000,United States Dollar,United States,210000,United States,Medium,2023,job title data engineer employment type fullti...,"[job, title, data, engineer, employment, type,...",13
1,Data Engineer,Full-Time,Senior,Expert,165000,United States Dollar,United States,165000,United States,Medium,2023,job title data engineer employment type fullti...,"[job, title, data, engineer, employment, type,...",13
2,Data Engineer,Full-Time,Senior,Expert,185900,United States Dollar,United States,185900,United States,Medium,2023,job title data engineer employment type fullti...,"[job, title, data, engineer, employment, type,...",13
3,Data Engineer,Full-Time,Senior,Expert,129300,United States Dollar,United States,129300,United States,Medium,2023,job title data engineer employment type fullti...,"[job, title, data, engineer, employment, type,...",13
4,Data Scientist,Full-Time,Senior,Expert,140000,United States Dollar,United States,140000,United States,Medium,2023,job title data scientist employment type fullt...,"[job, title, data, scientist, employment, type...",13


In [37]:
train_samples = len(train)
test_samples = len(test)

In [38]:
del train
del test
gc.collect()

0

In [39]:
train_test["full_text_sequence_length"].describe()

count    3300.000000
mean       13.282121
std         0.508892
min        13.000000
25%        13.000000
50%        13.000000
75%        14.000000
max        15.000000
Name: full_text_sequence_length, dtype: float64